In [ ]:
# cargamos un fichero con trazas, usando código de t-mon
from fileBrowserAndUploadButtonToLoadProcessStatements import load_players_info_from_file
out, err, xapiData =[],[], []
df=load_players_info_from_file("xapi-sg-sample-data-array.json", xapiData, out, err)
print(out, err)
#xapiData

In [ ]:
# cargamos pandas, plotly
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
df = pd.json_normalize(xapiData)
df

In [ ]:
df = pd.json_normalize(xapiData)

# algo de información básica
print(f"total of {len(df)} lines and {len(df['actor.name'].unique())} actors")

In [ ]:
import plotly.express as px
progress_verbs=["http://adlnet.gov/expapi/verbs/initialized", "http://adlnet.gov/expapi/verbs/progressed", "http://adlnet.gov/expapi/verbs/completed"]
progress_data=df.loc[df['verb.id'].isin(progress_verbs)].copy()
progress_data
# Define a function to update the progress for specific verbs
def update_progress(row):
    if row['verb.id'] == "http://adlnet.gov/expapi/verbs/initialized":
        return 0
    elif row['verb.id'] == "http://adlnet.gov/expapi/verbs/completed":
        return 1
    else:
        return row['result.extensions.https://w3id.org/xapi/seriousgames/extensions/progress']
progress_data['result.extensions.https://w3id.org/xapi/seriousgames/extensions/progress'] = progress_data.apply(update_progress, axis=1)

actors=progress_data['actor.name'].unique()

object_id_levels=progress_data['object.id'].unique()

for game in object_id_levels:
    bar_game_data=progress_data.loc[progress_data['object.id'] == game].copy()
    data = []
    data_init = []
    for actor in bar_game_data['actor.name'].unique():
       bar_data=bar_game_data.loc[bar_game_data['actor.name'] == actor].copy()
       data.append(go.Scatter(x=bar_data['timestamp'], y=bar_data['result.extensions.https://w3id.org/xapi/seriousgames/extensions/progress'], name=actor, hovertext=f"<b>{actor}</b>", mode="lines+markers"))
       first =bar_data.timestamp.min()
       bar_data.timestamp = pd.to_timedelta(pd.to_datetime(bar_data['timestamp']) - pd.to_datetime(first)) + pd.to_datetime('1970/01/01')
       data_init.append(go.Scatter(x=bar_data['timestamp'], y=bar_data['result.extensions.https://w3id.org/xapi/seriousgames/extensions/progress'], name=actor, hovertext=f"<b>{actor}</b>", mode="lines+markers"))
    #
    fig = go.Figure(
        layout_title_text='Progress during game level ' + str(game),
        data=data
    )
    fig.update_xaxes(categoryorder="total descending")
    fig.show()
    fig2 = go.Figure(
        layout_title_text='Progress (same origin) during game level ' + str(game),
        data=data_init
    )
    fig2.update_xaxes(categoryorder="total descending")
    fig2.show()

##Progress Data

In [ ]:
progress_game=progress_data.loc[~progress_data['object.id'].str.contains('Levels', case=False)]
# Filter for "started" and "completed"
progress_game_started = progress_game[progress_game['verb.id'].str.contains('initialized')]
progress_game_completed = progress_game[progress_game['verb.id'].str.contains('completed')]

# Merge to find actors who started and completed the same object
progress_game_merged = pd.merge(progress_game_started, progress_game_completed, on=['actor.name'], suffixes=('_started', '_completed'))

# Get the count of unique actors who started and completed the same object
actor_started_completed = progress_game_merged['actor.name'].unique()

# Find actors who have not completed the same object they started
progress_game_only_started = progress_game_started[~progress_game_started['actor.name'].isin(progress_game_completed['actor.name'])]

# Get the count of unique actors who only started an object
actor_only_started = progress_game_only_started['actor.name'].unique()

print(f"Number of actors who started and completed the same object: {len(actor_started_completed)}")
print(f"Number of actors who only started: {len(actor_only_started)}")

# Create data for pie chart
data = {
    'Category': ['Started and Completed', 'Only Started'],
    'Count': [len(actor_started_completed), len(actor_only_started)],
    'Actors': [', '.join(actor_started_completed), ', '.join(actor_only_started)]
}

df_pie = pd.DataFrame(data)

# Create pie chart with Plotly
fig = px.pie(df_pie, values='Count', names='Category', title='Actors by Activity Status', 
             hover_data=['Actors'], labels={'Actors': 'Actors'})

# Show the plot
fig.show()

In [ ]:
only_cutscenes = df[df['object.definition.type'].str.contains('cutscene', case=False)]

df_skipped = only_cutscenes[only_cutscenes['verb.id'].str.contains('skipped')]
df_accessed = only_cutscenes[only_cutscenes['verb.id'].str.contains('accessed')]
print("Skipped " + str(len(df_skipped)))
print("Access " + str(len(df_accessed)))
# Merge to find actors who start and skipped the same object
df_access_skipped = pd.merge(df_skipped, df_accessed, on=['object.id','actor.name'], how= 'outer', suffixes=('_skipped', '_accessed'))
df_access_skipped

# Group by 'object' and 'actor' to get the count
df_grouped = df_access_skipped[['object.id',  'actor.name', 'verb.id_skipped', 'verb.id_accessed']].groupby(['object.id', 'actor.name']).count().reset_index()
df_grouped['count_seen_but_skipped']=df_grouped['verb.id_skipped']
df_grouped['count_seen']=df_grouped['verb.id_accessed']-df_grouped['verb.id_skipped']

# Create a stacked bar chart with Plotly
fig = px.bar(df_grouped, x='object.id', y=['count_seen', 'count_seen_but_skipped'], 
             color_discrete_sequence=['#1f77b4', '#ff7f0e'],
             hover_name='actor.name', 
             labels={'count_seen_but_skipped': 'Access Count', 'count_seen': 'Seen Count', 'actor.name':'Actor'})
### Show the plot
fig.show()

### Intentemos ver puntuación en cada completable...

In [ ]:
df_completable= df[df['verb.id'].str.contains('completed')]
df_completable

score_columns = [col for col in df_completable.columns if 'score' in col]
if len(score_columns) > 0:
    print(score_columns[0])
    df_completable[score_columns[0]]

data = []
for c in df_completable['actor.name'].unique():
    bar_data = df_completable[df_completable['actor.name'] == c]
    data.append(go.Bar(x=bar_data['object.id'], y=bar_data[score_columns[0]], name=c))

fig = go.Figure(
    layout_title_text="Score by completable",
    data=data
)
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='group')
### Show the plot
fig.show()

In [ ]:
data = []
for c in df_completable['object.id'].unique():
    bar_data = df_completable[df_completable['object.id'] == c]
    data.append(go.Bar(x=bar_data['actor.name'], y=bar_data[score_columns[0]], name=c))
    
fig = go.Figure(
    layout_title_text="Score by completable",
    data=data
)
fig.update_xaxes(categoryorder="total descending")
fig.update_layout(barmode='group')
### Show the plot
fig.show()